## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-14-11-37-31 +0000,bbc,"Post Office could be owned by its workers, gov...",https://www.bbc.com/news/articles/czjk2eynz1lo
1,2025-07-14-11-22-10 +0000,bbc,First Indian astronaut to visit ISS heads back...,https://www.bbc.com/news/articles/ckgl13l3e9xo
2,2025-07-14-11-17-00 +0000,wsj,EU Trade Chief to Contact U.S. Officials After...,https://www.wsj.com/economy/trade/eu-trade-chi...
3,2025-07-14-10-57-17 +0000,nypost,Registered sex offender to walk free after ele...,https://nypost.com/2025/07/14/us-news/register...
4,2025-07-14-10-39-42 +0000,bbc,The striker with an 'icon' for a team-mate,https://www.bbc.com/sport/football/videos/cx2n...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
107,will,5
292,trump,5
13,trade,4
132,market,3
134,nyc,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
31,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...,32
33,2025-07-14-01-05-35 +0000,nypost,Trump announces US will send Patriot missiles ...,https://nypost.com/2025/07/13/world-news/trump...,24
19,2025-07-14-06-08-00 +0000,wsj,"China’s exports grew at a faster clip in June,...",https://www.wsj.com/economy/trade/chinas-expor...,22
40,2025-07-13-22-44-12 +0000,nypost,Ex-Secret Service director fires back at Rand ...,https://nypost.com/2025/07/13/us-news/ex-secre...,21
41,2025-07-13-22-43-30 +0000,nypost,Ex-AOC top aide urges Mamdani to run primaries...,https://nypost.com/2025/07/13/us-news/ex-aoc-a...,20


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
31,32,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...
41,20,2025-07-13-22-43-30 +0000,nypost,Ex-AOC top aide urges Mamdani to run primaries...,https://nypost.com/2025/07/13/us-news/ex-aoc-a...
33,19,2025-07-14-01-05-35 +0000,nypost,Trump announces US will send Patriot missiles ...,https://nypost.com/2025/07/13/world-news/trump...
17,17,2025-07-14-06-47-51 +0000,nypost,NYC fortune teller allegedly scammed 43-year-o...,https://nypost.com/2025/07/14/us-news/nyc-fort...
26,16,2025-07-14-04-47-53 +0000,nypost,Teddy bear purportedly wrapped in human skin l...,https://nypost.com/2025/07/14/us-news/teddy-be...
19,16,2025-07-14-06-08-00 +0000,wsj,"China’s exports grew at a faster clip in June,...",https://www.wsj.com/economy/trade/chinas-expor...
35,14,2025-07-13-23-10-41 +0000,nypost,Devoted dad gunned down en route to pick up da...,https://nypost.com/2025/07/13/us-news/nyc-dad-...
40,14,2025-07-13-22-44-12 +0000,nypost,Ex-Secret Service director fires back at Rand ...,https://nypost.com/2025/07/13/us-news/ex-secre...
10,13,2025-07-14-09-52-34 +0000,bbc,Body of former Nigerian leader Buhari to be fl...,https://www.bbc.com/news/articles/c93kxl08glwo
6,13,2025-07-14-10-07-58 +0000,nypost,"Gunman in Kentucky church shooting declared, ‘...",https://nypost.com/2025/07/14/us-news/gunman-i...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
